## スタッキング

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, RepeatedKFold, train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier

In [6]:
class MyStackingClassifierCV:
    
    def __init__(self, estimators, final_estimator, cv):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv
    
    def fit(self, X, y):
        #2層目のモデルに使用する学習データのデータフレーム
        pred_features = pd.DataFrame()
        
        # 1層目のモデル
        for model_name, model in self.estimators:
            pred_proba_df = pd.DataFrame()
            
            for n, train_index, test_index in enumerate(cv.split(X)):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                model.fit(X_train, y_train)
                y_pred_proba = model.predict_proba(X_test)[:, 1].to_list()
                pred_proba_df[n] = y_pred_proba
            pred_features[model_name] = pred_proba_df.mean(axis = 1)
        
        # 2層目のモデル
        self.

In [26]:
df = sns.load_dataset('titanic')
df = df.dropna()

# 学習データ作成
X = df.loc[:, (df.columns != 'survived') & (df.columns != 'alive')]
y = df["survived"]

# 決定木モデルではワンホットエンコーディングではなく、ラベルエンコーディングでも問題ない
# 理由：
oe = OrdinalEncoder()
oe.set_output(transform = 'pandas')
X = oe.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [55]:
class StackingClassifierCV():
    
    def __init__(self, estimators, final_estimator, cv):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv
    
    def fit(self, X, y):
        pred_features = {}
        #1層目のモデル学習
        for model_name, model in self.estimators:
            preds = []
            new_y = []
            
            for train_index, val_index in self.cv.split(X):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                model.fit(X_train, y_train)
                pred = model.predict_proba(X_val)[:, 1].tolist()
                preds += pred
                new_y += y_val.tolist()
            pred_features[model_name] = preds
            # 再度全データで学習
            model.fit(X, y) #全データで学習したモデルはself.estimatorsに格納される
        
        # 2層目のモデル学習
        new_X = pd.DataFrame(pred_features)
        self.final_estimator.fit(new_X, new_y)
    
    def predict_proba(self, X):
        # 1層目のモデルで特徴量(予測値)生成
        pred_features = {}
        for model_name, model in self.estimators:
            pred = model.predict_proba(X)[:,1]
            pred_features[model_name] = pred
        
        new_X = pd.DataFrame(pred_features)
        final_pred = self.final_estimator.predict_proba(new_X)
        
        return final_pred

In [56]:
df = sns.load_dataset('titanic')
df = df.dropna()

# 学習データ作成
X = df.loc[:, (df.columns != 'survived') & (df.columns != 'alive')]
y = df["survived"]

# 決定木モデルではワンホットエンコーディングではなく、ラベルエンコーディングでも問題ない
# 理由：
oe = OrdinalEncoder()
oe.set_output(transform = 'pandas')
X = oe.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

cv = KFold(n_splits = 5, shuffle = True, random_state = 0)
final_estimator = LogisticRegression()
stacking_cv = StackingClassifierCV(estimators = [('rf', RandomForestClassifier()), ('knn', KNeighborsClassifier())],
                                  final_estimator = final_estimator,
                                  cv = cv)
stacking_cv.fit(X_train, y_train)
y_pred_stacking_cv = stackin_cv.predict_proba(X_test)

In [57]:
from sklearn.metrics import roc_auc_score
print(f"stacking CV AUC: {roc_auc_score(y_test, y_pred_stacking_cv[:, 1])}")

stacking CV AUC: 0.8125
